## install transformers

In [1]:
! pip install transformers datasets evaluate seqeval accelerate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 M

## Import libraries

In [2]:
from datasets import load_dataset, load_metric, ClassLabel, Sequence
import random
import pandas as pd
import numpy as np
from IPython.display import display, HTML

import transformers
from transformers import AutoTokenizer, AlbertForTokenClassification, TrainingArguments, Trainer
from transformers import AlbertTokenizerFast

from transformers import DataCollatorForTokenClassification




## Load the dataset

In [4]:
data = load_dataset("conll2003")

  0%|          | 0/3 [00:00<?, ?it/s]

In [24]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [25]:
data['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

## We are using NER task for token classification

In [26]:
task = "ner"
model_checkpoint = "albert-base-v2"
batch_size = 16

In [27]:
label = data['train'].features[f"{task}_tags"].feature.names
print(label)

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


## just to see how dataset look like

In [28]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [29]:
show_random_elements(data['train'])

,id,tokens,pos_tags,chunk_tags,ner_tags
0,10476,"[CINCINNATI, AT, COLORADO]","[NNP, NNP, NNP]","[B-NP, I-NP, I-NP]","[B-ORG, O, B-LOC]"
1,1824,"[Relations, with, Russia, ,, which, is, our, main, partner, ,, have, great, importance, ,, "", Kuchma, said, .]","[NNS, IN, NNP, ,, WDT, VBZ, PRP$, JJ, NN, ,, VBP, JJ, NN, ,, "", NNP, VBD, .]","[B-NP, B-PP, B-NP, O, B-NP, B-VP, B-NP, I-NP, I-NP, O, B-VP, B-NP, I-NP, O, O, B-NP, B-VP, O]","[O, O, B-LOC, O, O, O, O, O, O, O, O, O, O, O, O, B-PER, O, O]"
2,409,"[He, added, :, "", If, no, one, asked, ,, I, never, opened, my, mouth, .]","[PRP, VBD, :, "", IN, DT, NN, VBD, ,, PRP, RB, VBD, PRP$, NN, .]","[B-NP, B-VP, O, O, B-SBAR, B-NP, I-NP, B-VP, O, B-NP, B-ADVP, B-VP, B-NP, I-NP, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,12149,"[--, Steve, Weizman, ,, Copenhagen, newsroom, +45, 33969650]","[:, NNP, NNP, ,, NNP, NN, CD, CD]","[B-NP, I-NP, I-NP, O, B-NP, I-NP, I-NP, I-NP]","[O, B-PER, I-PER, O, B-LOC, O, O, O]"
4,4506,"[70, 71, 74]","[CD, CD, CD]","[B-NP, I-NP, B-NP]","[O, O, O]"
5,4012,"[Graafschap, Doetinchem, 3, RKC, Waalwijk, 2]","[NNP, NNP, CD, NNP, NNP, CD]","[B-NP, I-NP, I-NP, I-NP, I-NP, I-NP]","[B-ORG, I-ORG, O, B-ORG, I-ORG, O]"
6,3657,"[Nottingham, Forest, 1, Middlesbrough, 1]","[NNP, NNP, CD, NNP, CD]","[B-NP, I-NP, I-NP, I-NP, I-NP]","[B-ORG, I-ORG, O, B-ORG, O]"
7,2286,"[69, 66, ,, Jose, Maria, Canizares, (, Spain, ), 67, 68, ,, Paul, Lawrie]","[CD, CD, ,, NNP, NNP, NNPS, (, NNP, ), CD, CD, ,, NNP, NNP]","[B-NP, I-NP, O, B-NP, I-NP, I-NP, O, B-NP, O, B-NP, I-NP, O, B-NP, I-NP]","[O, O, O, B-PER, I-PER, I-PER, O, B-LOC, O, O, O, O, B-PER, I-PER]"
8,12066,"[Hwa, Kay, plunges, on, rights, issue, ,, earnings, .]","[NNP, NNP, VBZ, IN, NNS, NN, ,, NNS, .]","[B-NP, I-NP, B-VP, B-PP, B-NP, I-NP, O, B-NP, O]","[B-PER, I-PER, O, O, O, O, O, O, O]"
9,1679,"[But, it, was, the, bullish, profit, forecast, for, 1996/97, that, took, the, spotlight, in, the, market, ,, with, some, analysts, saying, 20, percent, may, even, be, an, understatement, .]","[CC, PRP, VBD, DT, JJ, NN, NN, IN, CD, WDT, VBD, DT, NN, IN, DT, NN, ,, IN, DT, NNS, VBG, CD, NN, MD, RB, VB, DT, NN, .]","[O, B-NP, B-VP, B-NP, I-NP, I-NP, I-NP, B-PP, B-NP, B-NP, B-VP, B-NP, I-NP, B-PP, B-NP, I-NP, O, B-PP, B-NP, I-NP, B-VP, B-NP, I-NP, B-VP, I-VP, I-VP, B-NP, I-NP, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


## Lets preprocess the data

tokenize the data

In [30]:
tokenizer = AlbertTokenizerFast.from_pretrained(model_checkpoint)

In [31]:
# this will make sure the tokenizer is fast
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [32]:
## we need to preprocess the data since if our words already tokinze it will tokenize again wchich will end up giving more ids
## we will use truncation = true to reduce the size of the text that are bigger than maximum size and will use
## is_split_into_words=true then we align them
label_all_tokens = True # we can change the value of this flag to change the strategy to set the label for first token only
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], return_offsets_mapping=True, truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [33]:
## use map to apply the above function on all sentences
tokenize_data = data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

In [34]:
tokenize_data['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0],
 'input_ids': [2, 2898, 12170, 18, 548, 645, 20, 16617, 388, 8624, 13, 9, 3],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'offset_mapping': [[0, 0],
  [0, 2],
  [0, 6],
  [6, 7],
  [0, 6],
  [0, 4],
  [0, 2],
  [0, 7],
  [0, 7],
  [0, 4],
  [0, 1],
  [0, 1],
  [0, 0]],
 'labels': [-100, 3, 0, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]}

## fine tune the model

In [35]:
model = AlbertForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label))

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForTokenClassification: ['predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.decoder.bias', 'predictions.dense.bias', 'predictions.bias', 'predictions.dense.weight', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably

## Initiate training
### before that we need training arguments which will need folder name and other optional

In [36]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

## make all our example to same size 

In [37]:
data_collector = DataCollatorForTokenClassification(tokenizer)

## last we need metric from prediction, we use seqeval metric

In [38]:
metric = load_metric("seqeval")

## since the meric will take list of labels for predictions and reference we need to perform post preocess 


*   selecting predicted index for each token with max logit
*   cnvert it to string label
*   ignore everywhere we set label of -100 from earliear





In [39]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## we will only get overall accuracy/f1score/recall/precision


In [40]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenize_data["train"],
    eval_dataset=tokenize_data["validation"],
    data_collator=data_collector,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

## Now we train our trainer

In [42]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.168200,0.081442,0.930315,0.933850,0.932079,0.984991
2,0.058200,0.064895,0.947024,0.949493,0.948257,0.988044
3,0.028700,0.065191,0.954569,0.956347,0.955457,0.988768


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: -100 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 7 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarnin

TrainOutput(global_step=2634, training_loss=0.07274960131923991, metrics={'train_runtime': 461.3459, 'train_samples_per_second': 91.305, 'train_steps_per_second': 5.709, 'total_flos': 95347038073386.0, 'train_loss': 0.07274960131923991, 'epoch': 3.0})

In [43]:
trainer.evaluate()

{'eval_loss': 0.06519101560115814,
 'eval_precision': 0.9545691129452004,
 'eval_recall': 0.9563468414779499,
 'eval_f1': 0.955457150299557,
 'eval_accuracy': 0.9887680336249006,
 'eval_runtime': 13.566,
 'eval_samples_per_second': 239.569,
 'eval_steps_per_second': 15.038,
 'epoch': 3.0}

In [44]:
predictions, labels, _ = trainer.predict(tokenize_data["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'100': {'precision': 0.9579056775367026,
  'recall': 0.9589023280010405,
  'f1': 0.9584037436630704,
  'number': 7689},
 '_': {'precision': 0.9501043115438108,
  'recall': 0.952920662598082,
  'f1': 0.9515104030643337,
  'number': 5735},
 'overall_precision': 0.9545691129452004,
 'overall_recall': 0.9563468414779499,
 'overall_f1': 0.955457150299557,
 'overall_accuracy': 0.9887680336249006}